In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

True

# Problem parameters

In [2]:
# PARAMS

problem_name = 'CA_irv03_2_128_raw_weighted' # 'block_setting_classes_size-hairs_weighted-classes'

#dataset_dir = os.path.join(config.DATASET_DIR,'PH2','Clases','')
dataset_dir = os.path.join(config.DATASET_DIR,'ISIC_2020','Train','0128-0128_raw')

CoordAtt = True
Weighted_sampling = True

#lr = 1e-3
lr = 1e-5
n_epoch = 30
batch_size = 32

inverted_residual_setting_v1 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 1, 2],
        [6, 32, 2, 2],
        [6, 64, 3, 2],
        [6, 96, 2, 1],
        [6, 160, 2, 2],
        [6, 320, 1, 1],
    ]

inverted_residual_setting_v0 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

inverted_residual_setting = inverted_residual_setting_v1

# Original setting for mobilenet v2 (https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py)

# OTHER PARAMS
labels = [0, 1]
split_ratio = [0.75, 0.15, 0.1] # train, val, test
transform = transforms.Compose([
    transforms.ToTensor(),
#    transforms.Resize((128, 128))
])

criterion = torch.nn.CrossEntropyLoss()
log_dir = os.path.join('log',problem_name) 
model_dir = os.path.join('models',problem_name+'.pt')

# Dataset

In [3]:
# Derma dataset 
dataset = Derma(dataset_dir,labels=labels,transform=transform)

# Train-test splittings
#dataset.shuffle(manual_seed=42) 
train_set, val_set, test_set = dataset.split_rand(split_ratio=split_ratio,manual_seed=42)

In [4]:
# Weighted sampling
if Weighted_sampling:
    from derma.dataset import get_samples_weight
    train_sampler, train_weights = get_samples_weight(train_set)
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, sampler=train_sampler)
    val_sampler, val_weights = get_samples_weight(val_set)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=0, sampler=val_sampler)
    test_sampler, test_weights = get_samples_weight(test_set)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0, sampler=test_sampler)
else:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)


# Model

In [5]:
if CoordAtt:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
else:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2

# Training

In [6]:
from derma.utils import train_val
from torch.utils.tensorboard import SummaryWriter

#optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
tb_writer = SummaryWriter(log_dir=log_dir)

train_val(model,train_loader,val_loader,optimizer,criterion,tb_writer,n_epoch,model_dir=model_dir)


100%|██████████| 30/30 [36:27<00:00, 72.92s/epoch, vacc=0.6875, vls=14.8579]               


# Test

In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.load_state_dict(torch.load(model_dir))
    model.to(device)
else:
    model.load_state_dict(torch.load(model_dir), map_location=torch.device('cpu'))

In [9]:
from derma.experiment import test_experiment
metrics_df = test_experiment(model,test_loader)

In [10]:
#metrics_df.info()
metrics_df.head()

,Sensitivity,Specificity,Precission,Recall,Accuracy
0,1.0,0.5,0.5,1.0,0.666667
